In [1]:
#import libs
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.mask import mask
import numpy as np
import os
import zipfile
import geopandas as gpd


/tmp/ipykernel_11201/3327879154.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# Paths to your data
road_shapefile = 'roads_filtered.shp'
satellite_image_tif = 'satellite_imagery_testt.tif'
output_dir = 'Tileee_final'

# Create output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)


In [3]:
def process_tiles():
    if not os.path.isfile(road_shapefile):
        raise FileNotFoundError(f"Shapefile {road_shapefile} does not exist.")

    if not os.path.isfile(satellite_image_tif):
        raise FileNotFoundError(f"TIFF file {satellite_image_tif} does not exist.")
    
    # Load the shapefile
    roads = gpd.read_file(road_shapefile)
    roads['oneway'] = roads['oneway'].astype(str).str.lower()
    filtered_roads = roads[(roads['maxspeed'] >= 50) & (roads['oneway'] == 'f')]
    buffered_roads = filtered_roads.copy()
    buffered_roads['geometry'] = buffered_roads['geometry'].buffer(20)

    # Open the satellite image and apply mask
    with rasterio.open(satellite_image_tif) as src:
        img_crs = src.crs
        img_transform = src.transform
        img_data = src.read()
        img_meta = src.meta.copy()
        
        buffered_roads_geom = buffered_roads.geometry
        out_image, out_transform = mask(src, buffered_roads_geom, crop=True)
        
    # Reproject the image
    dst_crs = 'EPSG:25833'
    reprojected_data = np.empty_like(out_image)
    for i in range(out_image.shape[0]):
        reproject(
            source=out_image[i],
            destination=reprojected_data[i],
            src_transform=out_transform,
            src_crs=src.crs,
            dst_transform=img_transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )

    out_meta = img_meta.copy()
    out_meta.update({
        'crs': dst_crs,
        'transform': img_transform
    })

    # Define tile size and calculate number of tiles
    tile_size = 512
    n_bands, height, width = reprojected_data.shape

    # Calculate number of tiles needed to cover the full extent
    num_tiles_x = int(np.ceil(width / tile_size))
    num_tiles_y = int(np.ceil(height / tile_size))

    for i in range(num_tiles_y):
        for j in range(num_tiles_x):
            # Define tile bounds
            window = rasterio.windows.Window(
                col_off=j * tile_size,
                row_off=i * tile_size,
                width=min(tile_size, width - j * tile_size),
                height=min(tile_size, height - i * tile_size)
            )
            tile_data = reprojected_data[:, window.row_off:window.row_off + window.height, window.col_off:window.col_off + window.width]
            
            tile_meta = out_meta.copy()
            tile_meta.update({
                'height': tile_data.shape[1],
                'width': tile_data.shape[2],
                'transform': rasterio.windows.transform(window, img_transform)
            })

            tile_filename = os.path.join(output_dir, f'tile_{i * tile_size}_{j * tile_size}.tif')
            with rasterio.open(tile_filename, 'w', **tile_meta) as dest:
                dest.write(tile_data)
            
            print(f'Saved {tile_filename}')

    print('Tiling complete!')


In [4]:
def zip_tiles(output_dir):
    zip_file_path = os.path.join(output_dir, 'tiles.zip')
    with zipfile.ZipFile(zip_file_path, 'w') as zipf:
        for root, _, files in os.walk(output_dir):
            for file in files:
                if file.endswith('.tif'):
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, output_dir))
    print(f"All tiles have been saved and zipped into {zip_file_path}")

# Process and zip tiles
process_tiles()
zip_tiles(output_dir)


Saved Tileee_final/tile_0_0.tif
Saved Tileee_final/tile_0_512.tif
Saved Tileee_final/tile_0_1024.tif
Saved Tileee_final/tile_0_1536.tif
Saved Tileee_final/tile_0_2048.tif
Saved Tileee_final/tile_0_2560.tif
Saved Tileee_final/tile_0_3072.tif
Saved Tileee_final/tile_0_3584.tif
Saved Tileee_final/tile_0_4096.tif
Saved Tileee_final/tile_0_4608.tif
Saved Tileee_final/tile_0_5120.tif
Saved Tileee_final/tile_0_5632.tif
Saved Tileee_final/tile_0_6144.tif
Saved Tileee_final/tile_0_6656.tif
Saved Tileee_final/tile_0_7168.tif
Saved Tileee_final/tile_0_7680.tif
Saved Tileee_final/tile_0_8192.tif
Saved Tileee_final/tile_0_8704.tif
Saved Tileee_final/tile_0_9216.tif
Saved Tileee_final/tile_0_9728.tif
Saved Tileee_final/tile_512_0.tif
Saved Tileee_final/tile_512_512.tif
Saved Tileee_final/tile_512_1024.tif
Saved Tileee_final/tile_512_1536.tif
Saved Tileee_final/tile_512_2048.tif
Saved Tileee_final/tile_512_2560.tif
Saved Tileee_final/tile_512_3072.tif
Saved Tileee_final/tile_512_3584.tif
Saved Tileee